In [1]:
import pandas as pd
import time
from typing import Literal
from functools import partial
from typing import Optional
import os
from tqdm import tqdm

In [2]:
df = pd.read_pickle("./data/data_02.pkl.gz")
df

,text,uuid,int_0,int_1,int_2,int_3,int_4,int_5,int_6,int_7,...,float_10,float_11,float_12,float_13,float_14,float_15,float_16,float_17,float_18,float_19
0,"Write an article based on this ""A man has been...",227f72fe-fecc-4a8b-83fd-4c0bf79044b5,-1848,-7448,7011,-9271,-818,7723,7408,2765,...,1070.510375,-2632.194811,-1511.181933,3382.551305,2358.230930,6346.687782,7923.668912,7232.114931,-266.226245,-9490.335888
1,Answer the following question: - number is 54 ...,43f035e4-7905-4d4e-9b78-9ce1dbe3f15e,9485,3322,-210,4539,-5712,-6642,-6763,5408,...,9887.030489,1584.941985,1049.882522,-7057.736693,3311.705920,9250.504381,-5306.747252,4346.091141,3705.083462,586.077705
2,Produce a long descriptive sentence that uses ...,7d6cf004-4fc9-44b2-abbb-6aa6bb9e111f,-6568,-1429,7863,-5897,-4845,-9337,-4177,4800,...,122.453601,-6977.767998,-1533.505752,8333.095625,8700.795720,668.340118,4788.639416,-5222.894166,189.314977,-749.222275
3,Write a title for this article:\n\nArbitration...,1391998a-7260-4037-a886-fdba49282e8f,9467,-7638,5174,9196,-2378,9352,-4298,7949,...,5982.354362,-8063.945081,1541.905625,-9395.554755,1492.364621,7765.053363,6506.765269,2367.205007,-785.401680,-5978.024988
4,Read the following paragraph and determine if ...,5d1a24d4-e1d3-4e94-aabd-e58af037cfb3,-9609,-2289,-5844,3732,9806,-9914,3537,5583,...,4007.461801,-1500.406799,2546.834503,-3985.334523,8679.221751,6246.762728,1169.103626,-6382.974071,-2610.357519,4656.015420
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363486,"Q: Context: Dragon Ball Z (ドラゴンボール ゼット, Dorago...",e28c91e5-2115-473b-b31e-dd144a23ff5c,2524,-6655,7295,-8549,9114,-3688,-1763,5185,...,1708.789279,-9009.858048,7385.617783,-706.417619,-7424.619217,-2887.129320,-7133.300432,-9886.948402,520.655812,1172.371076
363487,My question is: At the beginning of an academi...,a520be4b-773c-4147-b06a-47a74687060d,-8329,-9144,-8182,-6010,-6693,-2829,-1539,-7076,...,9303.174804,-6311.456145,9012.214267,3875.721583,3114.162623,2359.850872,5780.628130,7284.598954,5170.040989,927.500877
363488,"Leo: Given the sentence ""A small child in wate...",4fde38d0-d92e-40f4-8dd1-d0499f1f04b0,1880,-7913,-8065,1328,-4772,7595,-993,3088,...,-5330.552098,3927.901481,-112.163506,-8667.423113,6856.692759,-1924.300261,-7216.503573,6365.602925,1535.116783,-8458.252345
363489,Explain simply why yes is the correct answer t...,fc592a4e-986b-4502-be87-39beee10c6a9,-9594,-3164,-306,9538,5491,9416,-2350,6787,...,-5342.596101,3930.096931,4469.888351,4613.697146,1460.793708,3206.175178,529.310585,-1578.960446,-8725.192612,-5633.208819


In [3]:
# total memory usage in bytes without index
# see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.memory_usage.html
mem_usage = df.memory_usage(index=False, deep=True).sum()
mem_usage

np.int64(823608914)

In [4]:
# total memory usage in m_bytes without index
mem_size_mb = mem_usage / 1024 / 1024
mem_size_mb

np.float64(785.454668045044)

In [5]:
def evaluate_performance(
        df,
        method: Literal["csv", "parquet"],
        compression: Optional[str] = None,
        compression_level: Optional[int] = None,
    ) -> dict:
    filename = f"./test_data/data.{method}"

    # set save_fn and read_fn
    save_fn = None
    read_fn = None
    additional_save_params = {}
    if method == "csv":
        if compression is not None:
            additional_save_params["compression"] = compression
        assert compression_level is None
        save_fn = partial(df.to_csv, index=False, path_or_buf=filename, **additional_save_params)
        read_fn = partial(pd.read_csv, filepath_or_buffer=filename, **additional_save_params)
    elif method == "parquet":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_parquet, index=False, engine="pyarrow", path=filename, **additional_save_params)
        read_fn = partial(pd.read_parquet, path=filename, engine="pyarrow")
    elif method == "feather":
        if compression is not None:
            additional_save_params["compression"] = compression
        if compression_level is not None:
            additional_save_params["compression_level"] = compression_level
        save_fn = partial(df.to_feather, path=filename, **additional_save_params)
        read_fn = partial(pd.read_feather, path=filename)
    assert save_fn is not None
    assert read_fn is not None


    # write
    start_time = time.time()
    save_fn()
    save_time = time.time() - start_time
    # print("save_time:", save_time)

    # read
    start_time = time.time()
    for _ in range(10):
        read_fn()
    read_time = (time.time() - start_time) / 10
    # print("read_time:", read_time)

    # get size
    file_size = os.path.getsize(filename) / 1024 / 1024
    # print("file_size:", file_size)

    # delete file
    os.remove(filename)

    result = {
        "method": method,
        "save_time": save_time,
        "read_time": read_time,
        "file_size": file_size,
    }

    for k, v in additional_save_params.items():
        result[k] = v

    return result


In [6]:
eval_results = []

In [7]:
# parquet, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:12<04:05, 12.94s/it]

{'method': 'parquet', 'save_time': 2.312278985977173, 'read_time': 1.0616801977157593, 'file_size': 276.7370548248291, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:25<03:53, 12.97s/it]

{'method': 'parquet', 'save_time': 2.5550568103790283, 'read_time': 1.043575406074524, 'file_size': 262.0364189147949, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:39<03:41, 13.03s/it]

...

{'method': 'parquet', 'save_time': 85.78946304321289, 'read_time': 0.9966506004333496, 'file_size': 223.1301155090332, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [11:30<01:18, 78.98s/it]

{'method': 'parquet', 'save_time': 93.06958389282227, 'read_time': 0.9918389081954956, 'file_size': 222.9604148864746, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [13:10<00:00, 39.53s/it]

{'method': 'parquet', 'save_time': 89.84644603729248, 'read_time': 0.9848781108856202, 'file_size': 222.9604148864746, 'compression': 'zstd', 'compression_level': 20}


In [8]:
# parquet, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "parquet", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:08<02:38,  8.34s/it]

{'method': 'parquet', 'save_time': 1.536238193511963, 'read_time': 0.6799933910369873, 'file_size': 375.51332664489746, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:16<02:26,  8.15s/it]

{'method': 'parquet', 'save_time': 1.4375081062316895, 'read_time': 0.6575443029403687, 'file_size': 375.51332664489746, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:29<02:59, 10.55s/it]

...

{'method': 'parquet', 'save_time': 33.36378717422485, 'read_time': 0.6724496126174927, 'file_size': 293.3767786026001, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [08:51<00:39, 39.36s/it]

{'method': 'parquet', 'save_time': 33.410090923309326, 'read_time': 0.6825837135314942, 'file_size': 293.3767786026001, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [09:32<00:00, 28.63s/it]

{'method': 'parquet', 'save_time': 34.173468828201294, 'read_time': 0.7123449087142945, 'file_size': 293.3767786026001, 'compression': 'lz4', 'compression_level': 20}


In [9]:
# feather, zstd
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="zstd", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:09<03:00,  9.50s/it]

{'method': 'feather', 'save_time': 1.6337261199951172, 'read_time': 0.7863580226898194, 'file_size': 274.08423042297363, 'compression': 'zstd', 'compression_level': 1}


 10%|█         | 2/20 [00:20<03:02, 10.14s/it]

{'method': 'feather', 'save_time': 2.02420973777771, 'read_time': 0.8554614067077637, 'file_size': 258.8376941680908, 'compression': 'zstd', 'compression_level': 2}


 15%|█▌        | 3/20 [00:30<02:58, 10.48s/it]

...

{'method': 'feather', 'save_time': 155.8468301296234, 'read_time': 0.7980259895324707, 'file_size': 200.37107276916504, 'compression': 'zstd', 'compression_level': 18}


 95%|█████████▌| 19/20 [16:59<02:20, 140.14s/it]

{'method': 'feather', 'save_time': 191.88540387153625, 'read_time': 0.7650203227996826, 'file_size': 199.14063453674316, 'compression': 'zstd', 'compression_level': 19}


100%|██████████| 20/20 [20:59<00:00, 62.97s/it] 

{'method': 'feather', 'save_time': 232.0722999572754, 'read_time': 0.7568066120147705, 'file_size': 190.44285774230957, 'compression': 'zstd', 'compression_level': 20}


In [10]:
# feather, lz4
for cl in tqdm(range(1, 21)):
    r = evaluate_performance(df, "feather", compression="lz4", compression_level=cl)
    print(r)
    eval_results.append(r)

  5%|▌         | 1/20 [00:06<02:06,  6.68s/it]

{'method': 'feather', 'save_time': 1.290541172027588, 'read_time': 0.5380644083023072, 'file_size': 382.2165927886963, 'compression': 'lz4', 'compression_level': 1}


 10%|█         | 2/20 [00:13<01:57,  6.55s/it]

{'method': 'feather', 'save_time': 1.2576570510864258, 'read_time': 0.5195846080780029, 'file_size': 382.2165927886963, 'compression': 'lz4', 'compression_level': 2}


 15%|█▌        | 3/20 [00:23<02:22,  8.36s/it]

...

{'method': 'feather', 'save_time': 30.710247039794922, 'read_time': 0.5314020156860352, 'file_size': 295.0932788848877, 'compression': 'lz4', 'compression_level': 18}


 95%|█████████▌| 19/20 [07:50<00:35, 35.36s/it]

{'method': 'feather', 'save_time': 30.61126208305359, 'read_time': 0.541713285446167, 'file_size': 295.0932788848877, 'compression': 'lz4', 'compression_level': 19}


100%|██████████| 20/20 [08:26<00:00, 25.34s/it]

{'method': 'feather', 'save_time': 30.8820641040802, 'read_time': 0.5315488815307617, 'file_size': 295.0932788848877, 'compression': 'lz4', 'compression_level': 20}


In [11]:
# parquet
for c in ["gzip", "snappy", "lz4", "zstd", "brotli", None]:
    r = evaluate_performance(df, "parquet", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'parquet', 'save_time': 21.936084985733032, 'read_time': 1.857405996322632, 'file_size': 261.4524908065796, 'compression': 'gzip'}
{'method': 'parquet', 'save_time': 1.4769291877746582, 'read_time': 0.756915807723999, 'file_size': 372.26979064941406, 'compression': 'snappy'}
{'method': 'parquet', 'save_time': 1.436607837677002, 'read_time': 0.6520306825637817, 'file_size': 375.51332664489746, 'compression': 'lz4'}
{'method': 'parquet', 'save_time': 1.6624550819396973, 'read_time': 0.8908697843551636, 'file_size': 276.7370548248291, 'compression': 'zstd'}
{'method': 'parquet', 'save_time': 18.984238862991333, 'read_time': 1.5739156007766724, 'file_size': 230.74532318115234, 'compression': 'brotli'}
{'method': 'parquet', 'save_time': 1.4593620300292969, 'read_time': 0.7634199619293213, 'file_size': 372.26979064941406}


In [12]:
# feather
for c in ["lz4", "zstd", "uncompressed", None]:
    r = evaluate_performance(df, "feather", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'feather', 'save_time': 1.2862539291381836, 'read_time': 0.5309521913528442, 'file_size': 382.2165927886963, 'compression': 'lz4'}
{'method': 'feather', 'save_time': 1.38962721824646, 'read_time': 0.7580457925796509, 'file_size': 274.08423042297363, 'compression': 'zstd'}
{'method': 'feather', 'save_time': 0.34214329719543457, 'read_time': 0.38784117698669435, 'file_size': 634.9402027130127, 'compression': 'uncompressed'}
{'method': 'feather', 'save_time': 1.252026081085205, 'read_time': 0.5269888877868653, 'file_size': 382.2165927886963}


In [13]:
# csv
for c in ['zip', 'gzip', 'bz2', 'zstd', 'xz', 'tar', None]:
    r = evaluate_performance(df, "csv", compression=c)
    print(r)
    eval_results.append(r)

{'method': 'csv', 'save_time': 32.36922287940979, 'read_time': 6.146654200553894, 'file_size': 298.97631645202637, 'compression': 'zip'}
{'method': 'csv', 'save_time': 36.42905068397522, 'read_time': 8.302969312667846, 'file_size': 298.6697826385498, 'compression': 'gzip'}
{'method': 'csv', 'save_time': 49.01720690727234, 'read_time': 20.00485780239105, 'file_size': 230.6348533630371, 'compression': 'bz2'}
{'method': 'csv', 'save_time': 14.664268970489502, 'read_time': 6.1076120138168335, 'file_size': 289.5586061477661, 'compression': 'zstd'}
{'method': 'csv', 'save_time': 331.15426301956177, 'read_time': 15.681923604011535, 'file_size': 212.4040298461914, 'compression': 'xz'}
{'method': 'csv', 'save_time': 11.371416807174683, 'read_time': 5.131657218933105, 'file_size': 691.07421875, 'compression': 'tar'}
{'method': 'csv', 'save_time': 11.353543043136597, 'read_time': 5.117451095581055, 'file_size': 691.0666332244873}


In [14]:
df = pd.DataFrame(eval_results)
df

,method,save_time,read_time,file_size,compression,compression_level
0,parquet,2.312279,1.061680,276.737055,zstd,1.0
1,parquet,2.555057,1.043575,262.036419,zstd,2.0
2,parquet,2.830313,1.025920,253.571889,zstd,3.0
3,parquet,2.799268,1.013424,251.290937,zstd,4.0
4,parquet,4.279723,1.023085,245.694203,zstd,5.0
...,...,...,...,...,...,...
92,csv,49.017207,20.004858,230.634853,bz2,NaN
93,csv,14.664269,6.107612,289.558606,zstd,NaN
94,csv,331.154263,15.681924,212.404030,xz,NaN
95,csv,11.371417,5.131657,691.074219,tar,NaN


In [15]:
df["comp_ratio"] = mem_size_mb / df["file_size"]
df

,method,save_time,read_time,file_size,compression,compression_level,comp_ratio
0,parquet,2.312279,1.061680,276.737055,zstd,1.0,2.838271
1,parquet,2.555057,1.043575,262.036419,zstd,2.0,2.997502
2,parquet,2.830313,1.025920,253.571889,zstd,3.0,3.097562
3,parquet,2.799268,1.013424,251.290937,zstd,4.0,3.125678
4,parquet,4.279723,1.023085,245.694203,zstd,5.0,3.196879
...,...,...,...,...,...,...,...
92,csv,49.017207,20.004858,230.634853,bz2,NaN,3.405620
93,csv,14.664269,6.107612,289.558606,zstd,NaN,2.712593
94,csv,331.154263,15.681924,212.404030,xz,NaN,3.697927
95,csv,11.371417,5.131657,691.074219,tar,NaN,1.136571


In [16]:
df.to_pickle("./data/test_results_03.pkl.gz", compression="gzip")